# Protest Classification Exercise - SOLUTION
## Complete Implementation with Two Models

**Models Used**: Logistic Regression and Random Forest

---

## Step 1: Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports complete")

/Users/dmatekenya/My Drive (dmatekenya@gmail.com)/TEACHING/AIMS-DSCBI/.venv-llms/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports complete


## Step 2: Load and Prepare Data

In [10]:
from pathlib import Path
# ==================
# SETUP INPUT
# ==================
DIR_DATA =  Path.cwd().parents[0].joinpath("data", "conflict")
FILE_PROTESTS = DIR_DATA / "protests_filtered.csv"

In [13]:
# Load data
df = pd.read_csv(FILE_PROTESTS)

# Combine notes and description
df['text'] = df['notes'] + ' ' + df['description']

# Check data
print(f"Dataset shape: {df.shape}")
print(f"\nCategories:")
print(df['category'].value_counts())
print(f"\nSample text: {df['text'].iloc[0][:150]}...")

Dataset shape: (224, 4)

Categories:
category
Livelihood (Prices, jobs and salaries)    64
Political/Security                        56
Business and legal                        42
Social                                    26
Public service delivery                   25
Climate and environment                   11
Name: count, dtype: int64

Sample text: On 30 June 2024, retirees of Tehran steel industries staged a protest in front of the Steel Servants Pension Organization's office in Tehran - Distric...


In [14]:
# Encode target labels
le = LabelEncoder()
y = le.fit_transform(df['category'])

print(f"Encoded labels: {np.unique(y)}")
print(f"Label mapping: {dict(enumerate(le.classes_))}")

Encoded labels: [0 1 2 3 4 5]
Label mapping: {0: 'Business and legal', 1: 'Climate and environment', 2: 'Livelihood (Prices, jobs and salaries)', 3: 'Political/Security', 4: 'Public service delivery', 5: 'Social'}


In [15]:
# Split data (80/20)
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df['text'].values,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Training samples: {len(X_train_text)}")
print(f"Test samples: {len(X_test_text)}")

Training samples: 179
Test samples: 45


## Step 3: Create TF-IDF Features

In [16]:
# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

print(f"TF-IDF shape: {X_train_tfidf.shape}")
print(f"Number of features: {X_train_tfidf.shape[1]}")

TF-IDF shape: (179, 1000)
Number of features: 1000


## Step 4: Create Embedding Features

In [17]:
# Load lightweight embedding model
print("Loading embedding model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

print("\nEncoding training data...")
X_train_embed = embedding_model.encode(X_train_text, show_progress_bar=True)

print("\nEncoding test data...")
X_test_embed = embedding_model.encode(X_test_text, show_progress_bar=True)

print(f"\nEmbedding shape: {X_train_embed.shape}")
print(f"Embedding dimensions: {X_train_embed.shape[1]}")

Loading embedding model...

Encoding training data...


Batches: 100%|██████████| 6/6 [00:01<00:00,  3.57it/s]



Encoding test data...


Batches: 100%|██████████| 2/2 [00:00<00:00,  8.82it/s]


Embedding shape: (179, 384)
Embedding dimensions: 384


## Step 5: Train Model 1 (Logistic Regression) with TF-IDF

In [18]:
# Train Logistic Regression with TF-IDF
print("Training Logistic Regression with TF-IDF...")
lr_tfidf = LogisticRegression(max_iter=1000, random_state=42)
lr_tfidf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred_lr_tfidf = lr_tfidf.predict(X_test_tfidf)
acc_lr_tfidf = accuracy_score(y_test, y_pred_lr_tfidf)
f1_lr_tfidf = f1_score(y_test, y_pred_lr_tfidf, average='weighted')

print(f"\n✅ Logistic Regression (TF-IDF)")
print(f"   Accuracy: {acc_lr_tfidf:.4f}")
print(f"   F1-Score: {f1_lr_tfidf:.4f}")

Training Logistic Regression with TF-IDF...

✅ Logistic Regression (TF-IDF)
   Accuracy: 1.0000
   F1-Score: 1.0000


## Step 6: Train Model 1 (Logistic Regression) with Embeddings

In [19]:
# Train Logistic Regression with Embeddings
print("Training Logistic Regression with Embeddings...")
lr_embed = LogisticRegression(max_iter=1000, random_state=42)
lr_embed.fit(X_train_embed, y_train)

# Predict and evaluate
y_pred_lr_embed = lr_embed.predict(X_test_embed)
acc_lr_embed = accuracy_score(y_test, y_pred_lr_embed)
f1_lr_embed = f1_score(y_test, y_pred_lr_embed, average='weighted')

print(f"\n✅ Logistic Regression (Embeddings)")
print(f"   Accuracy: {acc_lr_embed:.4f}")
print(f"   F1-Score: {f1_lr_embed:.4f}")

Training Logistic Regression with Embeddings...

✅ Logistic Regression (Embeddings)
   Accuracy: 0.8000
   F1-Score: 0.7479


## Step 7: Train Model 2 (Random Forest) with TF-IDF

In [20]:
# Train Random Forest with TF-IDF
print("Training Random Forest with TF-IDF...")
rf_tfidf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_tfidf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred_rf_tfidf = rf_tfidf.predict(X_test_tfidf)
acc_rf_tfidf = accuracy_score(y_test, y_pred_rf_tfidf)
f1_rf_tfidf = f1_score(y_test, y_pred_rf_tfidf, average='weighted')

print(f"\n✅ Random Forest (TF-IDF)")
print(f"   Accuracy: {acc_rf_tfidf:.4f}")
print(f"   F1-Score: {f1_rf_tfidf:.4f}")

Training Random Forest with TF-IDF...

✅ Random Forest (TF-IDF)
   Accuracy: 1.0000
   F1-Score: 1.0000


## Step 8: Train Model 2 (Random Forest) with Embeddings

In [22]:
# Train Random Forest with Embeddings
print("Training Random Forest with Embeddings...")
rf_embed = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_embed.fit(X_train_embed, y_train)

# Predict and evaluate
y_pred_rf_embed = rf_embed.predict(X_test_embed)
acc_rf_embed = accuracy_score(y_test, y_pred_rf_embed)
f1_rf_embed = f1_score(y_test, y_pred_rf_embed, average='weighted')

print(f"\n✅ Random Forest (Embeddings)")
print(f"   Accuracy: {acc_rf_embed:.4f}")
print(f"   F1-Score: {f1_rf_embed:.4f}")

Training Random Forest with Embeddings...

✅ Random Forest (Embeddings)
   Accuracy: 0.9111
   F1-Score: 0.8860


## Step 9: Compare All Results

In [23]:
# Create comparison dataframe
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Logistic Regression', 'Random Forest', 'Random Forest'],
    'Vectorization': ['TF-IDF', 'Embeddings', 'TF-IDF', 'Embeddings'],
    'Accuracy': [acc_lr_tfidf, acc_lr_embed, acc_rf_tfidf, acc_rf_embed],
    'F1-Score': [f1_lr_tfidf, f1_lr_embed, f1_rf_tfidf, f1_rf_embed]
})

print("\n" + "="*70)
print("FINAL RESULTS COMPARISON")
print("="*70)
print(results.to_string(index=False))
print("="*70)

# Find best model
best_idx = results['F1-Score'].idxmax()
print(f"\n🏆 BEST MODEL: {results.loc[best_idx, 'Model']} with {results.loc[best_idx, 'Vectorization']}")
print(f"   Accuracy: {results.loc[best_idx, 'Accuracy']:.4f}")
print(f"   F1-Score: {results.loc[best_idx, 'F1-Score']:.4f}")
print("="*70)


FINAL RESULTS COMPARISON
              Model Vectorization  Accuracy  F1-Score
Logistic Regression        TF-IDF  1.000000  1.000000
Logistic Regression    Embeddings  0.800000  0.747908
      Random Forest        TF-IDF  1.000000  1.000000
      Random Forest    Embeddings  0.911111  0.885989

🏆 BEST MODEL: Logistic Regression with TF-IDF
   Accuracy: 1.0000
   F1-Score: 1.0000


## Step 10: Detailed Analysis of Best Model

In [ ]:
# Get predictions from best model
best_model_name = results.loc[best_idx, 'Model']
best_vectorization = results.loc[best_idx, 'Vectorization']

# Select the correct predictions based on best model
if best_model_name == 'Logistic Regression' and best_vectorization == 'TF-IDF':
    y_pred_best = y_pred_lr_tfidf
elif best_model_name == 'Logistic Regression' and best_vectorization == 'Embeddings':
    y_pred_best = y_pred_lr_embed
elif best_model_name == 'Random Forest' and best_vectorization == 'TF-IDF':
    y_pred_best = y_pred_rf_tfidf
else:
    y_pred_best = y_pred_rf_embed

# Print detailed classification report
print(f"\nDetailed Classification Report - {best_model_name} ({best_vectorization})")
print("="*70)
print(classification_report(y_test, y_pred_best, target_names=le.classes_))

## Key Insights

### Expected Observations:

1. **Embeddings vs TF-IDF**: 
   - Embeddings usually perform better because they capture semantic meaning
   - TF-IDF is faster but only captures word frequency

2. **Model Comparison**:
   - Logistic Regression is typically faster and works well with high-dimensional data
   - Random Forest can capture non-linear patterns but may overfit on small datasets

3. **Class Imbalance**:
   - Some categories have fewer samples (e.g., Climate: 11 samples)
   - F1-score is more informative than accuracy for imbalanced data
   - Smaller classes may have lower precision/recall

4. **Performance Tips**:
   - With only 224 samples, simpler models (Logistic Regression) often work better
   - Embeddings help when training data is limited
   - Consider using class weights for imbalanced classes

## Bonus: Visualize Results

In [ ]:
import matplotlib.pyplot as plt

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
results.plot(x='Model', y='Accuracy', kind='bar', ax=ax1, color=['steelblue', 'lightcoral', 'mediumseagreen', 'orange'])
ax1.set_title('Accuracy Comparison', fontsize=14, fontweight='bold')
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_xticklabels(results['Vectorization'], rotation=45)
ax1.set_ylim(0, 1)
ax1.legend(['TF-IDF', 'Embeddings'])

# F1-Score comparison
results.plot(x='Model', y='F1-Score', kind='bar', ax=ax2, color=['steelblue', 'lightcoral', 'mediumseagreen', 'orange'])
ax2.set_title('F1-Score Comparison', fontsize=14, fontweight='bold')
ax2.set_ylabel('F1-Score', fontsize=12)
ax2.set_xticklabels(results['Vectorization'], rotation=45)
ax2.set_ylim(0, 1)
ax2.legend(['TF-IDF', 'Embeddings'])

plt.tight_layout()
plt.show()